In [17]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from collections import Counter
import os, sys
from nltk.tokenize import RegexpTokenizer
import numpy as np
import keras


data_path = 'ohsumed-first-20000-docs'
# data_path = 'ohsumed-all'
rep_train = 'training'
rep_test = 'test'

documents_train = {}
documents_test = {}

# words counting
word_counter = Counter()
for y in os.listdir(data_path + '/' + rep_train):
    documents_train[y] = 0
    for fichier in os.listdir(data_path + '/' + rep_train + '/' + y):
        documents_train[y] += 1
        with open(data_path + '/' + rep_train + '/' + y + '/' + fichier) as file:
            article = file.read().lower()
            tokenizer = RegexpTokenizer(r'\w+')
            words = tokenizer.tokenize(article)
            word_counter.update(words)
            
for y in os.listdir(data_path + '/' + rep_test):
    documents_test[y] = 0
    for fichier in os.listdir(data_path + '/' + rep_test + '/' + y):
        documents_test[y] += 1

#### Analyse des données

Taille du vocabulaires, fréquence des mots

In [11]:
print(len(word_counter))

30517


In [13]:
Counter(word_counter).most_common(10)

[('of', 191197),
 ('the', 190268),
 ('and', 128952),
 ('in', 124131),
 ('with', 69140),
 ('to', 68755),
 ('a', 68348),
 ('patients', 50174),
 ('was', 43078),
 ('were', 38666)]

In [15]:
print(documents_train)

{'C21': 546, 'C07': 100, 'C14': 1249, 'C08': 473, 'C04': 1163, 'C06': 588, 'C05': 283, 'C19': 191, 'C17': 295, 'C13': 281, 'C16': 200, 'C09': 125, 'C01': 423, 'C15': 215, 'C10': 621, 'C11': 162, 'C23': 1799, 'C20': 525, 'C18': 388, 'C03': 65, 'C02': 158, 'C22': 92, 'C12': 491}


In [18]:
print(documents_test)

{'C21': 717, 'C07': 146, 'C14': 1301, 'C08': 600, 'C04': 1467, 'C06': 632, 'C05': 429, 'C19': 191, 'C17': 348, 'C13': 386, 'C16': 228, 'C09': 129, 'C01': 506, 'C15': 320, 'C10': 941, 'C11': 202, 'C23': 2153, 'C20': 695, 'C18': 400, 'C03': 70, 'C02': 233, 'C22': 91, 'C12': 548}


In [ ]:
import matplotlib.pyplot as plt

dictionary = {1: 27, 34: 1, 3: 72, 4: 62, 5: 33, 6: 36, 7: 20, 8: 12, 9: 9, 10: 6, 11: 5, 
              12: 8, 2: 74, 14: 4, 15: 3, 16: 1, 17: 1, 18: 1, 19: 1, 21: 1, 27: 2}
plt.bar(, dictionary.values(),list(dictionary.keys()), color='g')
plt.show()